# Retail Demo Store - Neptune Workshop - Lab 1
Welcome to the Retail Demo Store Neptune Workshop. In this module we will be using [Amazon Rekognition](https://aws.amazon.com/rekognition/) to enrich product data and use that data to create a graph database with [Amazon Neptune](https://aws.amazon.com/neptune/). Over these progressive labs, we will show how a graph database can be integrated with [Amazon Personalize](https://aws.amazon.com/personalize/) and [Amazon OpenSearch Service](https://aws.amazon.com/opensearch-service/) to enhance user recommendations.
You should complete the labs in the order specified below.

## Workshop overview

### Prerequesites
This workshop requires that you have set up OpenSearch with the Retail Demo Store. You can do so by completing the OpenSearch lab (0-StartHere), or by building the index automatically as part of deployment (only applicable when creating the CloudFormation stack).

### Labs
- **Lab 1**: Introduction and data preparation (this lab) (_@todo Add time estimate_)
- **Lab 2**: Ingest data into Neptune and construct graph (_@todo Add time estimate_)
- **Lab 3**: Integrate Neptune with OpenSearch (_@todo Add time estimate_)

### Cleanup
The cleanup lab will tear down all of the Neptune resources created by the labs in this workshop.
- **Lab 4**: Cleanup resources (_@todo Add time estimate_)

## Setup
This workshop will be using the Python programming language and the AWS SDK for Python. Even if you are not fluent in Python, the code cells should be reasonably intuitive. In practice, you can use any programming language supported by the AWS SDK to complete the same steps from this workshop in your application environment.

### Update Dependencies
To get started, we need to perform a bit of setup. First, we need to ensure that a current version of botocore is currently installed. The botocore library is used by boto3, the AWS SDK library for Python.

The following cell will update pip and install the latest botocore library.

In [1]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade --no-deps --force-reinstall botocore

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.7 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 25.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.71
    Uninstalling botocore-1.29.71:
      Successfully uninstalled botocore-1.29.71


### Import Dependencies
Next we need to import some dependencies and libraries needed to complete this lab.

In [6]:
import boto3
import pandas as pd
import requests

### Create Clients
Next we need to create the AWS service clients needed for this workshop.
- **dynamodb**: This resource is used to get all the product information from the pre-existing DynamoDB table.
- **s3**: This client is used to fetch product images from S3 in order to pass them into Rekognition.
- **rekognition**: This client is used to generate labels from product images to enrich our product data.
- **ssm**: This client is used to access application configuration details stored in the Systems Manager parameter store.
- **servicediscovery**: this client is used to lookup the local IP addresses of the Retail Demo Store microservices that we'll need in the workshop

Finally we'll lookup an identifier, stored as a resource tag in the SageMaker instance at deployment time. We need this tag to lookup resources we need throughout the labs.

In [11]:
dynamodb = boto3.resource('dynamodb')
s3 = boto3.client('s3')
rekognition = boto3.client('rekognition')
ssm = boto3.client('ssm')
servicediscovery = boto3.client('servicediscovery')
sagemaker = boto3.client('sagemaker')

with open('/opt/ml/metadata/resource-metadata.json') as f:
    data = json.load(f)
    resource_arn = data["ResourceArn"]

response = sagemaker.list_tags(ResourceArn = resource_arn)
for tag in response['Tags']:
    if tag['Key'] == 'Uid':
        Uid = tag['Value']
assert Uid is not None, 'Uid could not be determined from notebook instance tags'
print('Uid:', Uid)

Uid: retaildemostore-eu-west-1


### Lookup DynamoDB table
When the Retail Demo Store stack was deployed in this account, a DynamoDB products table was created for you. The ARN of that table was stored in Systems Manager Parameter Store. Using the `ssm` boto3 client we created above, we can retrieve the table ARN.

In [16]:
table_arn = ssm.get_parameter(
    Name='retaildemostore-stack-products-table'
)
ddb_table = dynamodb.Table(table_arn)
#ddb_table = dynamodb.Table('retaildemostore-Base-13L7GWHV7GHLM-Tables-15EXRKL75UM4X-ProductsTable-1WOTHIE6CSX2U')

## Enrich Product Metadata with Rekognition
To improve the relationships stored by Neptune, the existing product data needs to be enriched as the current product descriptions don't give us a huge amount of information. We used Rekognition for this purpose.

Rekognition uses computer vision capabilities to extract information and images fromy our images and videos. As all the products in the Retail Demo Store have an image associated with them, we can use Rekognition to detect labels which give us insight into the product.

### Get S3 Objects
Before we can generate our image labels, we need to get the S3 bucket storing the images and perform a scan operation to get all the product information.

In [68]:
# Get the S3 bucket from ssm
bucketresponse = ssm.get_parameter(
    Name='retaildemostore-stack-bucket'
)
bucket = bucketresponse['Parameter']['Value']

response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='products',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

assert len(response['Instances']) > 0, 'Products service instance not found; check ECS to ensure it launched cleanly'

products_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Products Service Instance IP: {}'.format(products_service_instance))

response = requests.get('http://{}/products/all'.format(products_service_instance))
products = response.json()
products_df = pd.DataFrame(products)
pd.set_option('display.max_rows', 5)

products_df

Products Service Instance IP: 10.215.10.49


,id,url,sk,name,category,style,description,aliases,price,image,gender_affinity,current_stock,promoted,featured
0,e1669081-8ffc-4dec-97a6-e9176d7f6651,http://d34g24yoqfmrh4.cloudfront.net/#/product...,,Sans Pareil Scarf,apparel,scarf,Sans pareil scarf for women,None,124.99,http://d34g24yoqfmrh4.cloudfront.net/images/ap...,F,12,true,NaN
1,cfafd627-7d6b-43a5-be05-4c7937be417d,http://d34g24yoqfmrh4.cloudfront.net/#/product...,,Chef Knife,housewares,kitchen,A must-have for your kitchen,None,57.99,http://d34g24yoqfmrh4.cloudfront.net/images/ho...,NaN,9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463,12f93a36-e282-4445-92ae-356eb6a560fd,http://d34g24yoqfmrh4.cloudfront.net/#/product...,,Roses Arrangement,floral,arrangement,Roses arrangement grown sustainably on our org...,None,98.99,http://d34g24yoqfmrh4.cloudfront.net/images/fl...,NaN,19,NaN,NaN
2464,23e77743-ff28-400a-811f-e08d3cab5e45,http://d34g24yoqfmrh4.cloudfront.net/#/product...,,Ultrachic Scarf,apparel,scarf,Ultrachic scarf for women for every occasion,None,123.99,http://d34g24yoqfmrh4.cloudfront.net/images/ap...,F,7,NaN,NaN


Once we have all the variables needed, we can run the `detect_labels` operation for each product image and update the DynamoDB record with the new labels. The process may take a few minutes to complete.

In [92]:
import re

bucket = 'retaildemostore-base-13l7gwhv7ghlm-cl-webuibucket-4p3w695l7mc7'
for product in products_df.itertuples():
    
    # Construct s3 object key
    product_image_name = re.sub('^http:\/\/.*.cloudfront.net\/', '', product.image)
    
    obj = s3.get_object(Bucket=bucket, Key=product_image_name)
    
    # Get image labels
    response = rekognition.detect_labels(Image={'S3Object': {'Bucket': bucket, 'Name': product_image_name}},
                                         MaxLabels=10)

    # Format response so we only have the keys we need
    label_list = []
    for label in response['Labels']:
        label_list.append({
            'Name': label['Name'],
            'Confidence': label['Confidence']        
        })
    
    # Update in DynamoDB
    ddb_table.update_item(
    Key={'id': product.id},
    UpdateExpression="set image_labels = :image_labels",
    ExpressionAttributeValues={
        ':image_labels': json.dumps(label_list),
    },
)

### Test it
Once the labels have finished generating and being inserted into DynamoDB, we can view a few of the results.

Let's start by getting the a random product row and viewing the image.

In [106]:
from IPython.display import Image

test_product = products_df.iloc[10]

display(Image(url=test_product.image))

We can then query DynamoDB to get the image labels of our product.

In [107]:
response = ddb_table.get_item(Key={'id': test_product.id})

json.loads(response['Item']['image_labels'])

[{'Name': 'Accessories', 'Confidence': 100.0},
 {'Name': 'Glasses', 'Confidence': 100.0},
 {'Name': 'Cleaning', 'Confidence': 55.946510314941406},
 {'Name': 'Person', 'Confidence': 55.946510314941406},
 {'Name': 'Goggles', 'Confidence': 55.15962219238281}]

Compare each label with its level of confidence and see how it matches up. For the 10th product, Rekognition has 100% confidence that the image is both an accessory and a pair of glasses. 

Try changing the test product and see how results change.


## Lab 1 Summary
In this lab, you have used Rekognition to generate image labels for each product in the Retail Demo Store. In the next lab, you will ingest this new product data into Neptune and construct a graph from it.

### Store Variables Needed in the Next Lab
We will pass some variables initialized in this lab by storing them in the notebook environment.

In [108]:
%store ddb_table
%store bucket
%store Uid

PicklingError: Can't pickle <class 'boto3.resources.factory.dynamodb.Table'>: attribute lookup dynamodb.Table on boto3.resources.factory failed


### Continue to Lab 2
Open [Lab 2](./Lab-2-Construct-Graph-with-Neptune.ipynb) to continue the workshop.